In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import sys
import os
import numpy as np
import time
from imp import reload
import threading
import imageio
# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")

import logging

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE


In [2]:
from L1 import Controllers
from L2 import CameraControl
from L3 import SystemsBuilder
#from L4 import MicroscopeDisplay

import matplotlib.pyplot as plt
import time

reload(Controllers)
reload(CameraControl)
reload(SystemsBuilder)
#reload(MicroscopeDisplay)

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


<module 'L3.SystemsBuilder' from 'C:\\Users\\Luke\\Desktop\\Barracuda\\AutomatedCE\\L3\\SystemsBuilder.py'>

In [3]:
SYSTEM_CONFIG = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
sys = SystemsBuilder.CESystem()
sys.load_config(SYSTEM_CONFIG)


sys.open_controllers()
sys.startup_utilities()

['utility', 'ard1', 'pressure', 'outlet_pressure']
['utility', 'ard1', 'pressure', 'outlet_pressure']
['utility', 'mmc1', 'xy', 'xy_stage']
['utility', 'mmc1', 'xy', 'xy_stage']
['utility', 'mmc1', 'z', 'objective', 'offset', '0', 'default', '0', 'min_z', '0', 'max_z', '10', 'scale', '1', 'invert', '1']
['utility', 'mmc1', 'z', 'objective', 'offset', '0', 'default', '0', 'min_z', '0', 'max_z', '10', 'scale', '1', 'invert', '1']
['utility', 'ard1', 'z', 'outlet_z', 'offset', '300', 'default', '295', 'min_z', '100', 'max_z', '301', 'scale', '1', 'invert', '1']
['utility', 'ard1', 'z', 'outlet_z', 'offset', '300', 'default', '295', 'min_z', '100', 'max_z', '301', 'scale', '1', 'invert', '1']
['utility', 'ard2', 'z', 'inlet_z', 'offset', '25', 'default', '20', 'min_z', '-0.01', 'max_z', '24.25', 'scale', '1', 'invert', '1']
['utility', 'ard2', 'z', 'inlet_z', 'offset', '25', 'default', '20', 'min_z', '-0.01', 'max_z', '24.25', 'scale', '1', 'invert', '1']
['utility', 'ard1', 'rgb', 'inlet_

In [6]:
def get_z_stack(sys, folder, stop_flag, distance=100, step = 1):
    start_z = sys.objective.read_z()
    x, y= sys.xy_stage.read_xy()
    st = time.time()
    for i in range(-distance,distance, step):
        if stop_flag.is_set():
            break
        if i%10 == 0 :
            print(f"Step {int(i+(1*distance/step)):d} of {int(2*distance/step):d}, {time.time()-st:.1f} s elapsed, Imaging at {start_z+i/1000:.3f} mm")
        sys.objective.set_z((start_z+i/1000))
        time.sleep(0.15)
        z = sys.objective.read_z()
        name_tag = f"{int(z*1000 ):06d}z_{int(x*1000):06d}x_{int(y*1000):06d}y_{int(start_z*10000):06d}F.tiff"
        imageio.imwrite(os.path.join(folder,name_tag), sys.camera.snap())
    sys.objective.set_z(start_z)
    print("Finished")
    

In [21]:
DATA_OUTPUT = r"C:\Users\Luke\Pictures\Fl"
stop_flag = threading.Event()
threading.Thread(target=get_z_stack, args=(sys,DATA_OUTPUT,stop_flag,100,2)).start()

Step -50 of 100, 0.0 s elapsed, Imaging at 9.299 mm
Step -40 of 100, 3.6 s elapsed, Imaging at 9.309 mm
Step -30 of 100, 7.1 s elapsed, Imaging at 9.319 mm
Step -20 of 100, 10.6 s elapsed, Imaging at 9.329 mm
Step -10 of 100, 14.1 s elapsed, Imaging at 9.339 mm
Step 0 of 100, 17.6 s elapsed, Imaging at 9.349 mm
Step 10 of 100, 21.1 s elapsed, Imaging at 9.359 mm
Step 20 of 100, 24.7 s elapsed, Imaging at 9.369 mm
Step 30 of 100, 28.2 s elapsed, Imaging at 9.379 mm
Step 40 of 100, 31.7 s elapsed, Imaging at 9.389 mm
Step 50 of 100, 35.2 s elapsed, Imaging at 9.399 mm
Step 60 of 100, 38.7 s elapsed, Imaging at 9.409 mm
Step 70 of 100, 42.2 s elapsed, Imaging at 9.419 mm
Step 80 of 100, 45.7 s elapsed, Imaging at 9.429 mm
Step 90 of 100, 49.2 s elapsed, Imaging at 9.439 mm
Step 100 of 100, 52.7 s elapsed, Imaging at 9.449 mm
Step 110 of 100, 56.2 s elapsed, Imaging at 9.459 mm
Step 120 of 100, 59.8 s elapsed, Imaging at 9.469 mm
Step 130 of 100, 63.3 s elapsed, Imaging at 9.479 mm
Step 14

In [11]:
stop_flag.set()

Finished


In [7]:
ls = sys.lysis_laser

In [10]:
%matplotlib qt

In [20]:
ls.laser_standby()

In [19]:
ls.laser_stop()

In [21]:
for i in range(100):
    
    ls.laser_fire()
    time.sleep(0.4)

In [11]:
from L4 import DisplayWindows
dp = DisplayWindows.PLTMicroscopeDisplay(sys)
dp.show()

C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\skimage\exposure\exposure.py:351: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)


In [55]:
dp.live_image()

In [12]:
dp.single_image()

In [45]:
dp.gamma = 1.003

In [49]:
sys.camera.set_exposure(100)


True

In [12]:
z = sys.objective.read_z()

In [22]:
name_tag = f"{int(z*1000):06d}z_{int(x*1000):06d}x_{int(y*1000):05d}y_{start_z:05d}F"
print(name_tag)

001074z_002109x_01402y


In [81]:
img = sys.camera.snap()

In [24]:
img.dtype

dtype('>u2')

In [25]:
type(img)

numpy.ndarray

In [58]:
dp.gain=0.9

In [28]:
imageio.imwrite("test_image.tiff",img,)

In [59]:
dp.gamma = 0.6

In [80]:
sys.camera.set_exposure(1000)

True